# Mulitple Pixel Qinj Mapping

## Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hist import Hist
import datetime
import pandas as pd
from glob import glob
import mplhep as hep
hep.style.use("CMS")

### Board Name

In [ ]:
chip_name = "02D5_#12"

### Read data

In [ ]:
dir = '../../ETROC-Data/' + datetime.date.today().isoformat() + "_Array_Test_Results/"
files = glob(dir+'TID_testing_candidate*/*translated*')
files

### Save into pandas dataframe

In [ ]:
d = []

for ifile in files:
    with open(ifile, "r") as infile:    
        for line in infile.readlines():
            if line.split(' ')[2] == 'HEADER':
                bcid = line.strip().split(' ')[-1]
            elif line.split(' ')[2] == 'DATA':
                col = int(line.split(' ')[6])
                row = int(line.split(' ')[8])
                toa = int(line.split(' ')[10])
                tot = int(line.split(' ')[12])
                cal = int(line.split(' ')[14])
                d.append(
                    {
                    'col': col,
                    'row': row,
                    'toa': toa,
                    'tot': tot,
                    'cal': cal,     
                    'bcid': bcid,    
                    }
                )
            elif line.split(' ')[2] == 'TRAILER':
                pass
            
df = pd.DataFrame(d)

### filter data with BCID cut

In [ ]:
bcid_cuts = ['1854', '2087', '3522', '2699', '549', '527']

filtered_data = df[df['bcid'].isin(bcid_cuts)]
unique_values = filtered_data['bcid'].unique()
unique_values

### Delete big dataframe to save memory

In [ ]:
# Check the memory usage again
print(df.info())

# Check the memory usage again
print(filtered_data.info())
  
# Delete the old DataFrame
del df

## Make 2D map of toa, tot, cal

In [ ]:
cals = filtered_data.groupby(["col", "row"]).agg({'cal':['mean','std']})
cals.columns = cals.columns.droplevel()
cals.reset_index(inplace=True)

table_mean = cals.pivot_table(index='row', columns='col', values='mean')
table_mean = table_mean.round(1)

table_std = cals.pivot_table(index='row', columns='col', values='std')
table_std = table_std.round(2)

In [ ]:
plt.rcParams["xtick.major.size"] = 2.5
plt.rcParams["ytick.major.size"] = 2.5
plt.rcParams['xtick.minor.visible'] = False
plt.rcParams['ytick.minor.visible'] = False
plt.tight_layout()

fig, axes = plt.subplots(1, 2, figsize=(20, 14))

im1 = axes[0].imshow(table_mean, vmin=1)
im2 = axes[1].imshow(table_std, vmin=1)

axes[0].set_title('CAL Mean')
axes[1].set_title('CAL Std')

axes[0].set_xticks(np.arange(0,16))
axes[0].set_yticks(np.arange(0,16))
axes[1].set_xticks(np.arange(0,16))
axes[1].set_yticks(np.arange(0,16))

axes[0].invert_xaxis()
axes[0].invert_yaxis()
axes[1].invert_xaxis()
axes[1].invert_yaxis()

cbar1 = fig.colorbar(im1, ax=axes[0], shrink=0.6) 
cbar2 = fig.colorbar(im2, ax=axes[1], shrink=0.6)

for i in range(16):
    for j in range(16):
        if np.isnan(table_mean.iloc[i,j]): 
            continue
        axes[0].text(j, i, table_mean.iloc[i,j], ha="center", va="center", rotation=45, fontweight="bold", fontsize=8)

for i in range(16):
    for j in range(16):
        if np.isnan(table_std.iloc[i,j]): 
            continue
        axes[1].text(j, i, table_std.iloc[i,j], ha="center", va="center", rotation=45, color='white', fontweight="bold", fontsize=8)

fig.savefig('CAL_2D_Map.png')

In [ ]:
cals = filtered_data.groupby(["col", "row"]).agg({'toa':['mean','std']})
cals.columns = cals.columns.droplevel()
cals.reset_index(inplace=True)

table_mean = cals.pivot_table(index='row', columns='col', values='mean')
table_mean = table_mean.round(1)

table_std = cals.pivot_table(index='row', columns='col', values='std')
table_std = table_std.round(2)

In [ ]:
plt.rcParams["xtick.major.size"] = 2.5
plt.rcParams["ytick.major.size"] = 2.5
plt.rcParams['xtick.minor.visible'] = False
plt.rcParams['ytick.minor.visible'] = False
plt.tight_layout()

fig, axes = plt.subplots(1, 2, figsize=(20, 14))

im1 = axes[0].imshow(table_mean, vmin=1)
im2 = axes[1].imshow(table_std, vmin=1)

axes[0].set_title('TOA Mean')
axes[1].set_title('TOA Std')

axes[0].set_xticks(np.arange(0,16))
axes[0].set_yticks(np.arange(0,16))
axes[1].set_xticks(np.arange(0,16))
axes[1].set_yticks(np.arange(0,16))

axes[0].invert_xaxis()
axes[0].invert_yaxis()
axes[1].invert_xaxis()
axes[1].invert_yaxis()

cbar1 = fig.colorbar(im1, ax=axes[0], shrink=0.6) 
cbar2 = fig.colorbar(im2, ax=axes[1], shrink=0.6)

for i in range(16):
    for j in range(16):
        if np.isnan(table_mean.iloc[i,j]): 
            continue
        axes[0].text(j, i, table_mean.iloc[i,j], ha="center", va="center", rotation=45, fontweight="bold", fontsize=8)

for i in range(16):
    for j in range(16):
        if np.isnan(table_std.iloc[i,j]): 
            continue
        axes[1].text(j, i, table_std.iloc[i,j], ha="center", va="center", rotation=45, color='white', fontweight="bold", fontsize=8)

fig.savefig('TOA_2D_Map.png')

In [ ]:
cals = filtered_data.groupby(["col", "row"]).agg({'tot':['mean','std']})
cals.columns = cals.columns.droplevel()
cals.reset_index(inplace=True)

table_mean = cals.pivot_table(index='row', columns='col', values='mean')
table_mean = table_mean.round(1)

table_std = cals.pivot_table(index='row', columns='col', values='std')
table_std = table_std.round(2)

In [ ]:
plt.rcParams["xtick.major.size"] = 2.5
plt.rcParams["ytick.major.size"] = 2.5
plt.rcParams['xtick.minor.visible'] = False
plt.rcParams['ytick.minor.visible'] = False
plt.tight_layout()

fig, axes = plt.subplots(1, 2, figsize=(20, 14))

im1 = axes[0].imshow(table_mean, vmin=1)
im2 = axes[1].imshow(table_std, vmin=1)

axes[0].set_title('TOT Mean')
axes[1].set_title('TOT Std')

axes[0].set_xticks(np.arange(0,16))
axes[0].set_yticks(np.arange(0,16))
axes[1].set_xticks(np.arange(0,16))
axes[1].set_yticks(np.arange(0,16))

axes[0].invert_xaxis()
axes[0].invert_yaxis()
axes[1].invert_xaxis()
axes[1].invert_yaxis()

cbar1 = fig.colorbar(im1, ax=axes[0], shrink=0.6) 
cbar2 = fig.colorbar(im2, ax=axes[1], shrink=0.6)

for i in range(16):
    for j in range(16):
        if np.isnan(table_mean.iloc[i,j]): 
            continue
        axes[0].text(j, i, table_mean.iloc[i,j], ha="center", va="center", rotation=45, fontweight="bold", fontsize=8)

for i in range(16):
    for j in range(16):
        if np.isnan(table_std.iloc[i,j]): 
            continue
        axes[1].text(j, i, table_std.iloc[i,j], ha="center", va="center", rotation=45, color='white', fontweight="bold", fontsize=8)

fig.savefig('TOT_2D_Map.png')

## 2D map with individual histogram

In [ ]:
hcal = [[(
        Hist.new.Integer(135, 185, name="CAL", label="CAL [LSB]")
        .Int64()
      ) if not np.isnan(table_mean[i][j]) else np.nan
      for j in range(16)] 
     for i in range(16)]

htoa = [[(
        Hist.new.Integer(200, 270, name="TOA", label="TOA [LSB]")
        .Int64()
      ) if not np.isnan(table_mean[i][j]) else np.nan
      for j in range(16)] 
     for i in range(16)]

htot = [[(
        Hist.new.Integer(35, 85, name="TOT", label="TOT [LSB]")
        .Int64()
      ) if not np.isnan(table_mean[i][j]) else np.nan
      for j in range(16)] 
     for i in range(16)]

In [ ]:
for i in range(16):
    for j in range(16):
        cal_subset = filtered_data[(filtered_data['row'] == i) & (filtered_data['col'] == j)]['cal']
        toa_subset = filtered_data[(filtered_data['row'] == i) & (filtered_data['col'] == j)]['toa']
        tot_subset = filtered_data[(filtered_data['row'] == i) & (filtered_data['col'] == j)]['tot']
        if cal_subset.empty:
            continue
        else:
            hcal[i][j].fill(cal_subset)
            htoa[i][j].fill(toa_subset)
            htot[i][j].fill(tot_subset)

In [ ]:
# Create subplots for each unique combination of 'row' and 'col'
nrows = 16
ncols = 16

# Create a figure and subplots
fig, axs = plt.subplots(nrows, ncols, dpi=300, figsize=(160, 160))

for i in range(16):
    for j in range(16):
        if np.isnan(hcal[i][j]).any():
            axs[15-i][15-j].axis('off')
        else:
            hcal[i][j].project("CAL").plot1d(ax=axs[15-i][15-j], histtype="fill")
            hcal[i][j].project("CAL").plot1d(ax=axs[15-i][15-j], marker='.', histtype="errorbar", fillstyle='full', xerr=True, capsize=2, markersize=5)
            axs[15-i][15-j].set_title(f"Pixel ({i},{j}) CAL (LSB)", fontsize=22)
            axs[15-i][15-j].set_xticks(axs[15-i][15-j].get_xticks(), axs[15-i][15-j].get_xticklabels(), rotation='vertical')
            axs[15-i][15-j].minorticks_off()
            axs[15-i][15-j].set_xlim(np.argmax(hcal[i][j].values())-4, np.argmax(hcal[i][j].values())+4)

fig.savefig("CAL_chip_sim.pdf")
plt.close(fig)

In [ ]:
# Create subplots for each unique combination of 'row' and 'col'
nrows = 16
ncols = 16

# Create a figure and subplots
fig, axs = plt.subplots(nrows, ncols, dpi=300, figsize=(160, 160))

for i in range(16):
    for j in range(16):
        if np.isnan(htoa[i][j]).any():
            axs[15-i][15-j].axis('off')
        else:
            htoa[i][j].project("TOA").plot1d(ax=axs[15-i][15-j], histtype="fill")
            htoa[i][j].project("TOA").plot1d(ax=axs[15-i][15-j], marker='.', histtype="errorbar", fillstyle='full', xerr=True, capsize=2, markersize=5)
            axs[15-i][15-j].set_title(f"Pixel ({i},{j}) TOA (LSB)", fontsize=22)
            axs[15-i][15-j].set_xticks(axs[15-i][15-j].get_xticks(), axs[15-i][15-j].get_xticklabels(), rotation='vertical')
            axs[15-i][15-j].minorticks_off()
            axs[15-i][15-j].set_xlim(np.argmax(htoa[i][j].values())-4, np.argmax(htoa[i][j].values())+4)

fig.savefig("TOA_chip_sim.pdf")
plt.close(fig)

In [ ]:
# Create subplots for each unique combination of 'row' and 'col'
nrows = 16
ncols = 16

# Create a figure and subplots
fig, axs = plt.subplots(nrows, ncols, dpi=300, figsize=(160, 160))

for i in range(16):
    for j in range(16):
        if np.isnan(htot[i][j]).any():
            axs[15-i][15-j].axis('off')
        else:
            htot[i][j].project("TOT").plot1d(ax=axs[15-i][15-j], histtype="fill")
            htot[i][j].project("TOT").plot1d(ax=axs[15-i][15-j], marker='.', histtype="errorbar", fillstyle='full', xerr=True, capsize=2, markersize=5)
            axs[15-i][15-j].set_title(f"Pixel ({i},{j}) TOT (LSB)", fontsize=22)
            axs[15-i][15-j].set_xticks(axs[15-i][15-j].get_xticks(), axs[15-i][15-j].get_xticklabels(), rotation='vertical')
            axs[15-i][15-j].minorticks_off()
            axs[15-i][15-j].set_xlim(np.argmax(htot[i][j].values())-4, np.argmax(htot[i][j].values())+4)

fig.savefig("TOT_chip_sim.pdf")
plt.close(fig)